<a href="https://colab.research.google.com/github/TinaL9703/construction_defect_analysis/blob/data_extraction/CCCdata_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from PyPDF2 import PdfReader
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from os import listdir

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from os import listdir
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
listdir('/content/drive/MyDrive/DSP data processing/practise')

['_au_cases_vic_VCAT_2016_1790.pdf']

In [6]:
case = []
case_numbers = []
order = []
all_content = {'doc': [], 'vcat_no': [], 'case_no': [],
               'applicant': [], 'respondent': [], 'owner': [], 'builder': [],
               'catchwords': [], 'order': [], 'reason': []
              }
i = 1
for i,doc in enumerate(listdir('/content/drive/MyDrive/DSP data processing/pdf files')):
# for doc in ['_au_cases_vic_VCAT_2016_1790.pdf']:
#     print(doc)
    reader = PdfReader(f"/content/drive/MyDrive/DSP data processing/pdf files/{doc}")
    text = reader.pages[0].extract_text()+reader.pages[1].extract_text()+reader.pages[2].extract_text()
    text_list = []

    case_no = ''
    case_num = i
    section = {'doc': True, 'is_vcat_no': False, 'case_no':True, 
               'is_applicant': False, 'is_respondent': False, 'is_owner': False, 'is_builder': False,
               'is_catchwords': False, 'is_order': False, 'is_reason': False
              }
    case = {'doc': [], 'vcat_no': [], 'case_no': [], 
            'applicant': [], 'respondent': [], 'owner': [], 'builder': [],
            'catchwords': [], 'order': [], 'reason': []
           }
    order_found = False
    respondent_found = False

    for j in range(len(reader.pages)):  
      page = reader.pages[j]
      pagetext = page.extract_text()
      for no_, pageline in enumerate(pagetext.split('\n')):
        for i in pageline.strip().split('\n'):
          if i != '5KIPGF\x03D[\x03#WUV.++' or '4GVTKGXGF\x03HTQO\x03#WUV.' not in i:
            if i.strip() != "":
              text_list.append(i) #4GVTKGXGF\x03HTQO\x03#WUV.

      for line_no, line in enumerate(text.split('\n')):
#         print(line)
        line = line.strip()
        if len(line) == 0:
            continue
        new_section_detected = False
        if line.split(' ')[0].isupper() and not bool(re.search(r'[-\[\]\;]', line)):
            new_section_detected = True
            if new_section_detected and section['is_order']:
                order_found = True
            if new_section_detected and section['is_respondent']:
                respondent_found = True
#         print(new_section_detected)
        try:
            if not section['is_vcat_no']:
                if bool(re.search(r'([A-Z]{2}\s*\d+\/\d+)', line)) or bool(re.search(r'([A-Z]\s*\d+\/\d+)', line)):
                    section['is_vcat_no'] = True
                    if bool(re.search(r'[A-Z]{2}\s*\d+\/\d+', line)):
                        case['vcat_no'].append(re.search(r'([A-Z]{2}\s*\d+\/\d+)', line).group(0))
                    else:
                        case['vcat_no'].append(re.search(r'([A-Z]\s*\d+\/\d+)', line).group(0))
            elif not section['is_catchwords'] or \
            (not new_section_detected and section['is_catchwords'] and not section['is_applicant']) or \
            (new_section_detected and section['is_catchwords'] and case['catchwords'] == []):
                if bool(re.search(r'CATCHWORDS', line)):
                    section['is_catchwords'] = True
                else: 
                    case['catchwords'].append(line)

            elif not section['is_applicant'] or \
            (not new_section_detected and section['is_applicant'] and not section['is_respondent']) or \
            (new_section_detected and bool(re.search(r'APPLICANT', line)) and not section['is_respondent']):
                if bool(re.search(r'APPLICANT', line)):
                    section['is_applicant'] = True
                if not bool(re.search(r'APPLICANT', line)):
                    case['applicant'].append(line)
                else:
                    case['applicant'].append(re.search(r'APPLICANT[\:|S|\s](.*)', line).group(0))

            elif not section['is_respondent'] or \
            (not new_section_detected and section['is_respondent'] and not respondent_found) or \
            (new_section_detected and bool(re.search(r'RESPONDENT', line))):
                if bool(re.search(r'RESPONDENT', line)):
                    section['is_respondent'] = True
                if not bool(re.search(r'RESPONDENT', line)):
                    case['respondent'].append(line)
                else:
                    case['respondent'].append(re.search(r'RESPONDENT[S|\s](.*)', line).group(0))
            elif not section['is_order'] or \
            (not new_section_detected and section['is_order'] and not order_found):
                if bool(re.search(r'ORDER', line)) and not bool(re.search(r'DATE OF ORDER', line)):
                        section['is_order'] = True
                if len(line.split(' ')) > 1:
                    if bool(re.search('MEMBER|JUDGE|PRESIDENT', line.upper())) or bool(re.search('PAGE \d OF \d', line.upper())):
                        continue
                    if not new_section_detected and section['is_order']:
                        case['order'].append(line)
                else:
                    continue
            elif not section['is_owner']:
                if bool(re.search(r'\(.*OWNER.*\)', line.upper())):
#                     print(re.search(r'(.*\s.*)\s\(.*OWNER.*\)', line.upper()).group(0))
                    section['is_owner'] = True
                    case['owner'].append(re.search(r'(.{0,20})\s\(.*OWNER.*\)', line.upper()).group(0))
            elif not section['is_builder']:
                if bool(re.search(r'\(.*BUILDER.*\)', line.upper())):
                    section['is_builder'] = True
                    case['builder'].append(re.search(r'(.{0,20})\s\(.*BUILDER.*\)', line.upper()).group(0))

            #elif not section['is_reason'] and 'REASON' in line:
             #   section['is_reason'] = True
              #  continue # set flag and continue to next line

            #if section['is_reason']:
             #   case['reason'].append(line)
        except:
            continue
    if all(list(section.values())) or line_no + 1 == len(text.split('\n')):
        all_content['doc'].append(doc)
        all_content['vcat_no'].append(' '.join(case['vcat_no']))
        all_content['case_no'].append(case_num)
        all_content['applicant'].append(' '.join(case['applicant']))
        all_content['respondent'].append(' '.join(case['respondent']))
        all_content['owner'].append(' '.join(case['owner']))
        all_content['builder'].append(' '.join(case['builder']))
        all_content['catchwords'].append(' '.join(case['catchwords']))
        all_content['order'].append(' '.join(case['order']))
        #all_content['reason'].append(' '.join(case['reason']))
          
          
    for k in range(len(text_list)):
      if "REASON" in text_list[k] and "REASONABLE" not in text_list[k]:
        reason_index = k
        #case['reason'] = text_list[reason_index+1:]
    #trylist = "".join(text_list[reason_index+1:])
    all_content['reason'].append(''.join(text_list[reason_index+1:]))

In [9]:
all_content_df = pd.DataFrame(all_content)

In [10]:
all_content_df

,doc,vcat_no,case_no,applicant,respondent,owner,builder,catchwords,order,reason
0,_au_cases_vic_VCAT_2020_1073.pdf,BP1414/2019,0,APPLICANT John Beveridge,RESPONDENT Integrated Construction Management...,,,Who was the builder? Is a contractor liable fo...,The application is dismissed.,1 This is an application by John Bev eridge (t...
1,_au_cases_vic_VCAT_2012_284.pdf,D768/2009,1,RESPONDENT Karen Visser AND: DOMESTIC BUILDIN...,RESPONDENT Karen Visser RESPONDENT Karen Vis...,,,APPLICANT Verve Constructions Pty Ltd (ACN 13...,1 The Respondent must pay the Applicant a sum ...,Introduction1 Mrs Visser owns a block of land ...
2,_au_cases_vic_VCAT_2010_1325.pdf,D172/9,2,APPLICANT Imagination Developments Pty Ltd,"RESPONDENT Pink Painter Pty Ltd (trading as ""...",,,Domestic B uilding - applicant engaged the re...,1 The respondent pay the applicant the sum of ...,Introduction3 The subject matter of this proce...
3,_au_cases_vic_VCAT_2022_1156.pdf,BP224/2019,3,APPLICANT Shamrock Cabinets Pty Ltd (ACN:160 ...,RESPONDENT Catherine Hutchings RESPONDENT Ca...,,,DOMESTIC BUILDING CONTRACT – major domestic bu...,1 The respondents are liable to pay the applic...,THE DISPUTE1 The applicant (Shamrock) manufact...
4,_au_cases_vic_VCAT_2018_1558.pdf,BP365/2018,4,APPLICANT Gozul Pty Ltd (ACN 133 484 189),RESPONDENT Resources Consulting Services Pty ...,,,"RETAIL LEASES , whether ten ant in breach enti...",1. The landlord Resources Consulting Services ...,1 This is an application by Gozul Pty Ltd (th...
...,...,...,...,...,...,...,...,...,...,...
499,_au_cases_vic_VCAT_2014_1282.pdf,D1266/2012,499,APPLICANT OWNERS CORPORATION NO 1,PS611203E RESPONDENT FURMAN CONSTRUCTIONS PTY...,,,19 unit residential town -house development -v...,"1. For the reasons provided, the respondent m...",1 The applicant owns the common property of a ...
500,_au_cases_vic_VCAT_2022_224.pdf,BP1640/2019,500,APPLICANT Cameron Upton APPLICANT Lisa Upton,RESPONDENT Hartman Construction & Development...,1 THE APPLICANTS (THE OWNERS ),BRUNSWICK (THE HOUSE ). THE FIRST R ESPONDENT...,DOMESTIC BUILDING –– defective works –– damag...,"For the reasons set out in writing, the Tribun...",Background1 The Applicants (the Owners ) are t...
501,Copy of _au_cases_vic_VCAT_2020_1407.pdf,BP1071/2019,501,APPLICANT Aaron Ng APPLICANT Eric Lau,RESPONDENT Romas Corporation Pty Ltd ACN 082 ...,,,Domestic building contract – proceeding determ...,1. The respondent must pay to the applicant s ...,1 This proceeding came before me for a hearing...
502,Copy of _au_cases_vic_VCAT_1998_1 (3).pdf,D824/97,502,,,,,VICTORIAN CIVIL & ADMINISTRATIVE TRIBUNAL Vict...,,These proceedings relate to a domestic buildin...


In [11]:
def find_builder(field, prmy_a_field, prmy_r_field):
    if 'APPLICANT' in field.upper():
        return 'APPLICANT'
    elif 'RESPONDENT' in field.upper():
        return 'RESPONDENT'
    elif 'PTY' in prmy_a_field.upper() or 'LTD' in prmy_a_field.upper():
        return 'APPLICANT'
    elif 'PTY' in prmy_r_field.upper() or 'LTD' in prmy_r_field.upper():
        return 'RESPONDENT'
    else:
        return ''

def find_owner(o_field, b_field, prmy_a_field, prmy_r_field):
    if 'APPLICANT' in o_field.upper():
        return 'APPLICANT'
    elif 'RESPONDENT' in o_field.upper():
        return 'RESPONDENT'
    elif b_field != '' and b_field == 'APPLICANT':
        return 'RESPONDENT'
    elif b_field != '' and b_field == 'RESPONDENT':
        return 'APPLICANT'
    elif o_field != '' and set(o_field.upper().split(' ')) & set(prmy_r_field.upper().split(' ')):
        return 'RESPONDENT'
    elif o_field != '' and set(o_field.upper().split(' ')) & set(prmy_a_field.upper().split(' ')):
        return 'APPLICANT'
    else:
        return ''
all_content_df['builder'] = [find_builder(i[0],i[1],i[2]) for i in zip(all_content_df['builder'], 
                                                                        all_content_df['applicant'], 
                                                                        all_content_df['respondent'])]
all_content_df['owner'] = [find_owner(i[0],i[1],i[2],i[3]) for i in zip(all_content_df['owner'], 
                                                                        all_content_df['builder'],
                                                                        all_content_df['applicant'], 
                                                                        all_content_df['respondent'])]

In [12]:
all_content_df

,doc,vcat_no,case_no,applicant,respondent,owner,builder,catchwords,order,reason
0,_au_cases_vic_VCAT_2020_1073.pdf,BP1414/2019,0,APPLICANT John Beveridge,RESPONDENT Integrated Construction Management...,APPLICANT,RESPONDENT,Who was the builder? Is a contractor liable fo...,The application is dismissed.,1 This is an application by John Bev eridge (t...
1,_au_cases_vic_VCAT_2012_284.pdf,D768/2009,1,RESPONDENT Karen Visser AND: DOMESTIC BUILDIN...,RESPONDENT Karen Visser RESPONDENT Karen Vis...,RESPONDENT,APPLICANT,APPLICANT Verve Constructions Pty Ltd (ACN 13...,1 The Respondent must pay the Applicant a sum ...,Introduction1 Mrs Visser owns a block of land ...
2,_au_cases_vic_VCAT_2010_1325.pdf,D172/9,2,APPLICANT Imagination Developments Pty Ltd,"RESPONDENT Pink Painter Pty Ltd (trading as ""...",RESPONDENT,APPLICANT,Domestic B uilding - applicant engaged the re...,1 The respondent pay the applicant the sum of ...,Introduction3 The subject matter of this proce...
3,_au_cases_vic_VCAT_2022_1156.pdf,BP224/2019,3,APPLICANT Shamrock Cabinets Pty Ltd (ACN:160 ...,RESPONDENT Catherine Hutchings RESPONDENT Ca...,RESPONDENT,APPLICANT,DOMESTIC BUILDING CONTRACT – major domestic bu...,1 The respondents are liable to pay the applic...,THE DISPUTE1 The applicant (Shamrock) manufact...
4,_au_cases_vic_VCAT_2018_1558.pdf,BP365/2018,4,APPLICANT Gozul Pty Ltd (ACN 133 484 189),RESPONDENT Resources Consulting Services Pty ...,RESPONDENT,APPLICANT,"RETAIL LEASES , whether ten ant in breach enti...",1. The landlord Resources Consulting Services ...,1 This is an application by Gozul Pty Ltd (th...
...,...,...,...,...,...,...,...,...,...,...
499,_au_cases_vic_VCAT_2014_1282.pdf,D1266/2012,499,APPLICANT OWNERS CORPORATION NO 1,PS611203E RESPONDENT FURMAN CONSTRUCTIONS PTY...,APPLICANT,RESPONDENT,19 unit residential town -house development -v...,"1. For the reasons provided, the respondent m...",1 The applicant owns the common property of a ...
500,_au_cases_vic_VCAT_2022_224.pdf,BP1640/2019,500,APPLICANT Cameron Upton APPLICANT Lisa Upton,RESPONDENT Hartman Construction & Development...,APPLICANT,RESPONDENT,DOMESTIC BUILDING –– defective works –– damag...,"For the reasons set out in writing, the Tribun...",Background1 The Applicants (the Owners ) are t...
501,Copy of _au_cases_vic_VCAT_2020_1407.pdf,BP1071/2019,501,APPLICANT Aaron Ng APPLICANT Eric Lau,RESPONDENT Romas Corporation Pty Ltd ACN 082 ...,APPLICANT,RESPONDENT,Domestic building contract – proceeding determ...,1. The respondent must pay to the applicant s ...,1 This proceeding came before me for a hearing...
502,Copy of _au_cases_vic_VCAT_1998_1 (3).pdf,D824/97,502,,,,,VICTORIAN CIVIL & ADMINISTRATIVE TRIBUNAL Vict...,,These proceedings relate to a domestic buildin...
